In [12]:
import pandas as pd
import re
from concurrent.futures import ThreadPoolExecutor

import numpy as np
import pandas as pd
from astropy.coordinates import SkyCoord
from astropy.time import Time
from astroquery.jplhorizons import Horizons


# df = pd.read_csv('D:\Project\Earthquake\Comets\\raw_asteroid\AsteroidCT.csv')
df = pd.read_csv('D:\Project\Earthquake\Comets\\raw_asteroid\CLEAR101_P10.csv')            # round 2
df['Time'] = pd.to_datetime(df['Time'])

In [15]:
def calculate_position(obj, time):
    """
    Args: 
        obj: obj name str
        time: time (utc) format yyyy-mm-dd
    """
    try:
        epoch = Time(time)
        q = Horizons(obj, location='@0', epochs=epoch.tdb.jd)
        tab = q.vectors(refplane='earth')
        c = SkyCoord(tab['x'].quantity, tab['y'].quantity, tab['z'].quantity,
                     representation_type='cartesian', frame='icrs',
                     obstime=epoch)
        result = str(c)
        matches = re.findall(r'-?\d+\.\d+', result)
        result_list = [float(match) for match in matches]
        return result_list
    except Exception as e:
        print(f"Error calculating position for object {obj} at time {time}")
        return [99,99,99]

no_use = ['-74', '-49','-49', '-490', '-255' , '2000016', '20065803', '-64']

def process_chunk(chunk):
    chunk[['X_new_1', 'Y_new_1', 'Z_new_1']] = chunk.apply(
        lambda row: pd.Series(calculate_position(row['Object_Name'], row['Time'])), axis=1)
    return chunk


no_use = ['-74', '-49','-49', '-490', '-255' , '2000016', '20065803', '-64']

In [16]:

df = df[(df['X'] == 99) & (df['Y'] == 99) & (df['Z'] == 99) & (~df['Object_Name'].isin(no_use))]
df

,Time,Object_Name,X,Y,Z,X_new_1,Y_new_1,Z_new_1
2,2010-01-25,Orus,99.0,99.0,99.0,99.000000,99.000000,99.000000
3,2010-05-17,Dinkinesh,99.0,99.0,99.0,-1.690998,1.029076,0.513363
4,2011-08-16,399,99.0,99.0,99.0,0.801318,-0.564307,-0.244634
5,2012-07-11,SUN,99.0,99.0,99.0,99.000000,99.000000,99.000000
8,2015-04-27,Apophis,99.0,99.0,99.0,99.000000,99.000000,99.000000
9,2016-06-18,Annefrank,99.0,99.0,99.0,99.000000,99.000000,99.000000
10,2016-07-31,Dinkinesh,99.0,99.0,99.0,99.000000,99.000000,99.000000
11,2016-08-02,Leucus,99.0,99.0,99.0,99.000000,99.000000,99.000000


In [17]:
num_threads = 8
chunks = np.array_split(df, num_threads)

# Process each part concurrently using 8 threads
with ThreadPoolExecutor(max_workers=num_threads) as executor:
    results = list(executor.map(process_chunk, chunks))


df = pd.concat(results)

df.to_csv('CLEAR101_P11.csv', index=False)

Error calculating position for object Orus at time 2010-01-25 00:00:00


Error calculating position for object Dinkinesh at time 2016-07-31 00:00:00
Error calculating position for object Leucus at time 2016-08-02 00:00:00
Error calculating position for object SUN at time 2012-07-11 00:00:00


In [24]:
def calculate_position(obj, time):
    """
    Args : 
        obj     : obj name str
        time    : time (utc) format yyyy-mm-dd
    """
    epoch = Time(time)
    q = Horizons(obj, location='@0', epochs=epoch.tdb.jd)
    tab = q.vectors(refplane='earth')
    c = SkyCoord(tab['x'].quantity, tab['y'].quantity, tab['z'].quantity,
                representation_type='cartesian', frame='icrs',
                obstime=epoch)
    result = str(c)
    matches = re.findall(r'-?\d+\.\d+', result)
    result_list = [float(match) for match in matches]
    return result_list


calculate_position('Leucus','2016-08-02')

[-1.15388714, -5.06098388, -2.0440798]

Bring 99 out of DataSet

In [81]:
import pandas as pd


file1 = pd.read_csv('D:\Project\Earthquake\Comets\\raw_asteroid\AsteroidCT.csv')

file2 = pd.read_csv('D:\Project\Earthquake\Comets\\raw_asteroid\merged_file11_final.csv')

file1['Time'] = pd.to_datetime(file1['Time'])
file2['Time'] = pd.to_datetime(file2['Time'])

# merged_data = pd.merge(file1, file2, on=['Time', 'Object_Name'], how='left')


# merged_data.rename(columns={'X_y': 'X_file2', 'Y_y': 'Y_file2', 'Z_y': 'Z_file2'}, inplace=True)

# merged_data.drop(['X_x', 'Y_x', 'Z_x'], axis=1, inplace=True)

# merged_data.to_csv('merged_file1.csv', index=False)

# print(merged_data)


In [82]:
merged_df = pd.merge(file1, file2, on=['Object_Name', 'Time'], how='left', suffixes=('_file1', '_file2'))

merged_df['X_file1'] = merged_df['X_file2'].where(merged_df['X_file1'] == 99, merged_df['X_file1'])
merged_df['Y_file1'] = merged_df['Y_file2'].where(merged_df['Y_file1'] == 99, merged_df['Y_file1'])
merged_df['Z_file1'] = merged_df['Z_file2'].where(merged_df['Z_file1'] == 99, merged_df['Z_file1'])

result_df = merged_df.drop(['X_file2', 'Y_file2', 'Z_file2'], axis=1)

result_df.to_csv('AsteroidPosition.csv')

CleaR Asteroid NaN Value

In [89]:
import pandas as pd

df = pd.read_csv('D:\Project\Earthquake\Comets\\raw_asteroid\AsteroidPosition.csv')
# Delete NaN

df = df.dropna()
df.to_csv('AsteroidPositionFinal.csv')

In [91]:
import pandas as pd

df = pd.read_csv('D:\Project\Earthquake\Comets\\raw_asteroid\AsteroidPositionFinal.csv')
df

,Unnamed: 0,Time,Object_Name,X,Y,Z
0,0,1/1/2000,SUN,-0.007140,-0.002644,-0.000921
1,1,1/1/2000,199,-0.147852,-0.400633,-0.198918
2,2,1/1/2000,299,-0.725770,-0.039682,0.027895
3,3,1/1/2000,399,-0.175677,0.886197,0.384434
4,4,1/1/2000,301,-0.177800,0.884616,0.384015
...,...,...,...,...,...,...
287350,342415,1/14/2024,90000855,1.809491,-2.762580,-1.142571
287351,342416,1/14/2024,Dinkinesh,-1.767012,-1.412305,-0.643992
287352,342417,1/14/2024,20065803,-1.513543,-1.529300,-0.600365
287353,342418,1/14/2024,Gaspra,-1.826124,-1.613584,-0.794328


In [93]:
object_planet = ['SUN', '199', '299', '399', '301', '599', '699', '433', 'Eros', 'Orus', '90000188', '90000190', '90000191', 
                        'Leucus', 'Eurybates','Polymele', 'Vesta', 'Mathilde', 'Lutetia', 'Donaldjohanson','Braille', 'Annefrank', 
                        'Bennu', 'Itokawa', 'Apophis', 'Ryugu', '-122911', 'Patroclus', 
                        '90000855', 'Dinkinesh','Gaspra', 'Ceres']

df = df[df['Object_Name'].isin(object_planet)]
df

,Unnamed: 0,Time,Object_Name,X,Y,Z
0,0,1/1/2000,SUN,-0.007140,-0.002644,-0.000921
1,1,1/1/2000,199,-0.147852,-0.400633,-0.198918
2,2,1/1/2000,299,-0.725770,-0.039682,0.027895
3,3,1/1/2000,399,-0.175677,0.886197,0.384434
4,4,1/1/2000,301,-0.177800,0.884616,0.384015
...,...,...,...,...,...,...
287349,342414,1/14/2024,Patroclus,3.840027,-0.954550,-2.147478
287350,342415,1/14/2024,90000855,1.809491,-2.762580,-1.142571
287351,342416,1/14/2024,Dinkinesh,-1.767012,-1.412305,-0.643992
287353,342418,1/14/2024,Gaspra,-1.826124,-1.613584,-0.794328


In [95]:
df.to_csv('AsteroidPositionFinal1.csv')